# HopSlot - Deep Learning Model

## Import modules and dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
disease_df = pd.read_csv("./datasets/diseases_dataset.csv")
patient_df = pd.read_csv("./datasets/patient_data.csv")

## Define Paths

In [3]:
from pathlib import Path

BASE_PATH_ML = Path('./models/disease_ml')
BASE_PATH_ML.mkdir(parents=True, exist_ok=True)

VECTORIZED_MODEL_PATH = BASE_PATH_ML / 'symptoms_vectorizer.pkl'
MODEL_SAVE_PATH = BASE_PATH_ML / 'diseases_rfc_model.pkl'

BASE_PATH_DL = Path('./models/appointment_dl')
BASE_PATH_DL.mkdir(parents=True, exist_ok=True)

MODEL_SAVE_PATH_DL = BASE_PATH_DL / 'appointment_model.pth'

In [4]:
BASE_DATASET_PATH = Path('./datasets/synt')
BASE_DATASET_PATH.mkdir(parents=True, exist_ok=True)

DOCTOR_DATASET_PATH = BASE_DATASET_PATH / 'doctor_availability.csv'
PATIENT_DATASET_PATH = BASE_DATASET_PATH / 'patient_data.csv'

In [5]:
import dill

# Load the vectorizer
with open(VECTORIZED_MODEL_PATH, 'rb') as f:
    loaded_vectorizer = dill.load(f)

# Load the model
with open(MODEL_SAVE_PATH, 'rb') as f:
    loaded_model = dill.load(f)

In [6]:
sample_symptoms = loaded_vectorizer.transform([['fever', 'sore throat', 'cough']])

sample_pred = loaded_model.predict(sample_symptoms)

print(sample_pred)

['measles']


In [7]:
doctor_df = pd.read_csv(DOCTOR_DATASET_PATH).drop(columns=['Unnamed: 0'])

doctor_df

,doc_id,date,time_start,time_end,time_slot_start,time_slot_end,time_slot_duration,appointments_count,overtime_delay,event_type,event_time_stamp,delay_caused_due_to_event
0,d0611be3-30af-46f7-8dad-09cdee7a8d02,2024-02-07,9:00:00,11:00:00,9:00:00,9:15:00,15,4,6,NaN,NaN,NaN
1,7d9bdfee-82bd-465d-92f2-fa7517784bac,2024-01-22,9:00:00,11:00:00,9:15:00,9:30:00,15,4,9,NaN,NaN,NaN
2,b1077fe8-00dd-4f18-8cd2-5136ebb244d4,2024-01-30,9:00:00,11:00:00,9:00:00,9:15:00,15,4,13,NaN,NaN,NaN
3,0f5c3f46-c657-42eb-a85f-d9d588587d46,2024-02-05,18:00:00,20:00:00,18:45:00,19:00:00,15,4,4,NaN,NaN,NaN
4,cfdd52b1-09e6-426a-a289-5ea02ad7c2fd,2024-03-18,9:00:00,11:00:00,9:00:00,9:15:00,15,4,13,cannot make it,2024-03-18 00:00:00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,f8629e26-8080-4971-bc75-97066dc07a72,2024-03-13,9:00:00,11:00:00,9:00:00,9:15:00,15,4,10,NaN,NaN,NaN
996,6579926b-3b36-4f73-89af-abac9ed6d060,2024-02-18,20:00:00,23:00:00,20:00:00,20:15:00,15,6,8,cannot make it,2024-01-14 00:00:00,6.0
997,87188e10-5bd7-4395-9c66-985972e8d5a4,2024-03-17,20:00:00,23:00:00,20:30:00,20:45:00,15,6,7,NaN,NaN,NaN
998,b01e299f-e0a4-4e5c-a479-a56316fca581,2024-02-22,18:00:00,20:00:00,18:15:00,18:30:00,15,4,5,NaN,NaN,NaN


In [8]:
patient_df = pd.read_csv(PATIENT_DATASET_PATH)

patient_df

,Patient_Name,Patient_ID,Age,Cause_of_Visit,No_of_Previous_Visits,Cause_Matches_Previous_Visit,Doctor_Availability,Work_Shift_Time,Schedule_date_time,Delayed_by,Severity_of_Ailment
0,Arjun Patel,P0001,45,"['change in bowel habits', 'raised', 'physical...",4,No,Available,Evening,2024-11-28T07:20:38,0,Severe
1,Avani Kumar,P0002,48,"['pain in the lower right abdomen', 'runny nos...",5,No,Unavailable,Evening,2024-04-25T16:54:40,10,Mild
2,Dhruv Gupta,P0003,65,"['pain in the upper right abdomen', 'increased...",0,Yes,Available,Evening,2024-01-21T16:49:33,0,Severe
3,Aditi Singh,P0004,68,"['pain in the lower back or side', 'itchy', 'f...",3,Yes,Unavailable,Evening,2024-06-08T06:27:27,0,Severe
4,Advik Sharma,P0005,68,"['condition that affects the lungs', 'raisedse...",2,No,Unavailable,Afternoon,2024-09-21T09:33:23,5,Moderate
...,...,...,...,...,...,...,...,...,...,...,...
995,Aditi Singh,P0996,6,"['cough', 'damage to the kidneys that can lead...",9,Yes,Unavailable,Morning,2024-03-04T22:27:33,15,Severe
996,Advik Patel,P0997,39,"['elevated body temperature ', 'hypothyroidism...",3,No,Unavailable,Evening,2024-08-21T00:56:49,0,Severe
997,Aarav Chauhan,P0998,39,"['pain', 'excessive worry', 'memory loss']",8,Yes,Available,Afternoon,2024-02-03T19:54:02,15,Mild
998,Diya Reddy,P0999,66,['difficulty gettingcondition in which the hea...,2,No,Unavailable,Afternoon,2024-10-30T01:36:36,0,Severe
